In [ ]:
import os

def process_dataset_and_create_symlink(dataset_path):
    """
    Проверяет, что указанный путь существует, ищет в нём структуру COCO
    (папки annotations, train2017, val2017), и создаёт символическую ссылку
    в папке ./YOLOX/datasets/COCO, указывающую на найденную структуру.

    :param dataset_path: Путь к директории, в которой ожидается COCO-структура
    """
    if not os.path.exists(dataset_path):
        raise ValueError(f"Dataset path does not exist: {dataset_path}")

    # Автоматически ищем структуру COCO внутри указанной директории
    def find_coco_structure(base_dir):
        for root, dirs, files in os.walk(base_dir):
            if "annotations" in dirs and "train2017" in dirs and "val2017" in dirs:
                return root
        return None

    coco_root = find_coco_structure(dataset_path)
    if not coco_root:
        raise ValueError(
            "Provided directory does not contain the expected COCO structure "
            "(annotations/, train2017/, val2017/)."
        )

    print(f"Found COCO structure at: {coco_root}")

    # Путь к символической ссылке
    coco_symlink = "./YOLOX/datasets/COCO"
    coco_dir = os.path.dirname(coco_symlink)

    # Создаем путь к папке, если он не существует
    if not os.path.exists(coco_dir):
        os.makedirs(coco_dir)
        print(f"Created directory: {coco_dir}")

    # Если символическая ссылка или папка с таким именем уже существует — удаляем
    if os.path.islink(coco_symlink):
        os.unlink(coco_symlink)
        print(f"Removed old symbolic link: {coco_symlink}")
    elif os.path.exists(coco_symlink):
        os.rmdir(coco_symlink)
        print(f"Removed old directory: {coco_symlink}")

    # Создаем новую символическую ссылку на COCO-структуру
    os.symlink(coco_root, coco_symlink)
    print(f"Created symbolic link: {coco_symlink} -> {coco_root}")

In [ ]:
# Пример использования
coco_directory_path = "/workspace/shared/your-dataset-path"  # Замените на путь к вашей директории
process_dataset_and_create_symlink(coco_directory_path)

### Parameters for YOLOX Training Command

- **`-n <model>`**: Model name, e.g., `yolox-nano`, `yolox-s`, etc. YOLOX supports various models:
  - `yolox-nano`
  - `yolox-tiny`
  - `yolox-s`
  - `yolox-m`
  - `yolox-l`
  - `yolox-x`
- **`-d`**: Number of GPUs to use (e.g., `1` for a single GPU).
- **`-b`**: Batch size. Adjust based on available GPU memory.
- **`--fp16`**: Enables mixed-precision training.
- **`--cache ram`**: Caches the dataset in RAM for faster data loading.
- **`--ckpt`**: Path to a checkpoint file for fine-tuning, e.g., `/workspace/shared/YOLOX_outputs/yolox_pretrained.pth`.
- **`--resume`**: Resume training from a specific checkpoint.
- **`--start_epoch`**: Specify the starting epoch when resuming training.
- **`-expn <experiment_name>`**: Specifies the experiment name. This will be used to create a subdirectory in `output_dir` for storing results. For example, `-expn "yolox_nano_rusal_ppe"` will create a folder named `yolox_nano_rusal_ppe` in the `output_dir`.

### Positional Parameters (at the end of the command)
- **`max_epoch`**: Total number of training epochs (e.g., `300`).
- **`print_interval`**: Frequency of logging progress (e.g., every `50` iterations).
- **`eval_interval`**: Frequency of evaluation during training (e.g., every `30` epochs).
- **`output_dir`**: Path to save training results. Recommended path: `/workspace/shared/YOLOX_outputs` for compatibility with Windows (`C:\shared`).

In [ ]:
!python3 -m yolox.tools.train -expn "yolox_nano_rusal_ppe" -n yolox-nano -d 1 -b 64 --fp16 -o --cache ram max_epoch 300 print_interval 50 eval_interval 30 